In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/course-title/data.csv


In [2]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ["KAGGLE_USERNAME"] = user_secrets.get_secret("KAGGLE_KEY")
os.environ["KAGGLE_KEY"] = user_secrets.get_secret("KAGGLE_USERNAME")

In [3]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


In [4]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [5]:
import keras
import keras_nlp
import pandas as pd

2024-03-25 14:11:12.665464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 14:11:12.665570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 14:11:12.953810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
df = pd.read_csv('/kaggle/input/course-title/data.csv')
df.head()

,Unnamed: 0,course_title,course_organization,course_difficulty,lang,Key_ideas
0,1,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,Intermediate,en,Define causal effects using potential outcomes...
1,2,A Crash Course in Data Science,Johns Hopkins University,Mixed,en,How to describe the role data science plays in...
2,3,A Law Student's Toolkit,Yale University,Mixed,en,"Strategies for actively reading statutes, case..."
3,4,A Life of Happiness and Fulfillment,Indian School of Business,Mixed,en,The concept and importance of happiness and we...
4,5,ADHD: Everyday Strategies for Elementary Students,University at Buffalo,Beginner,en,How to understand and support children with AD...


In [7]:
df.dropna(axis=0, how='any', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 579 entries, 0 to 581
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           579 non-null    int64 
 1   course_title         579 non-null    object
 2   course_organization  579 non-null    object
 3   course_difficulty    579 non-null    object
 4   lang                 579 non-null    object
 5   Key_ideas            579 non-null    object
dtypes: int64(1), object(5)
memory usage: 31.7+ KB


In [8]:
def gen_prompt(example):
  return f"Instruction: Your task is to choose a suitable title for a course which teach these key ideas: {example['Key_ideas']}, the course's title should be in language {example['lang']}\nResponse:{example['course_title']}"

In [9]:
data = list(df.apply(gen_prompt, axis=1))
del df

In [10]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [11]:
prompt = "Instruction: Your task is to choose a suitable title for a course which teach these key ideas: Define causal effects using potential outcomes; Describe the difference between association and causation; Express assumptions with causal graphs; Implement several types of causal inference methods (e.g. matching, instrumental variables, inverse probability of treatment weighting); Identify which causal assumptions are necessary for each type of statistical method, the course's title should be in language en\nResponse:"
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction: Your task is to choose a suitable title for a course which teach these key ideas: Define causal effects using potential outcomes; Describe the difference between association and causation; Express assumptions with causal graphs; Implement several types of causal inference methods (e.g. matching, instrumental variables, inverse probability of treatment weighting); Identify which causal assumptions are necessary for each type of statistical method, the course's title should be in language en
Response: I am writing an academic research paper, and I am having trouble deciding which course I should write about. The course is called "Causal Inference in Health and Economics". I am not sure whether this is the right choice, as I am not sure whether I am writing about causal inference at all. Can you please advise me which course I should choose? Thank you.

Answer:

1. Define causal effects using potential outcomes.
2. Describe the difference between association and causation.
3.

In [12]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [13]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=2, batch_size=1)

Epoch 1/2
579/579 ━━━━━━━━━━━━━━━━━━━━ 790s 1s/step - loss: 0.4964 - sparse_categorical_accuracy: 0.4978
Epoch 2/2
579/579 ━━━━━━━━━━━━━━━━━━━━ 733s 1s/step - loss: 0.3108 - sparse_categorical_accuracy: 0.6516


In [14]:
prompt = "Instruction: Your task is to choose a suitable title for a course which teach these key ideas: Define causal effects using potential outcomes; Describe the difference between association and causation; Express assumptions with causal graphs; Implement several types of causal inference methods (e.g. matching, instrumental variables, inverse probability of treatment weighting); Identify which causal assumptions are necessary for each type of statistical method, the course's title should be in language en\nResponse:"
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction: Your task is to choose a suitable title for a course which teach these key ideas: Define causal effects using potential outcomes; Describe the difference between association and causation; Express assumptions with causal graphs; Implement several types of causal inference methods (e.g. matching, instrumental variables, inverse probability of treatment weighting); Identify which causal assumptions are necessary for each type of statistical method, the course's title should be in language en
Response:The Art of Causal Inference: A Graphical Approach


In [15]:
!pip install huggingface_hub

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
# Finetuned model
FINETUNED_MODEL_DIR = "./my_model"
FINETUNED_WEIGHTS_PATH = f"{FINETUNED_MODEL_DIR}/model.weights.h5"
FINETUNED_VOCAB_PATH = f"{FINETUNED_MODEL_DIR}/vocabulary.spm"

# Converted model
HUGGINGFACE_MODEL_DIR = "./Course_Title_Gemma_2b"

In [18]:
# Make sure the directory exists
%mkdir -p $FINETUNED_MODEL_DIR

gemma_lm.save_weights(FINETUNED_WEIGHTS_PATH)

gemma_lm.preprocessor.tokenizer.save_assets("./my_model")

In [19]:
del gemma_lm

In [20]:
# Download the conversion script from KerasNLP tools
!wget -nv -nc https://raw.githubusercontent.com/keras-team/keras-nlp/master/tools/gemma/export_gemma_to_hf.py

# Run the conversion script
# Note: it uses the PyTorch backend of Keras (hence the KERAS_BACKEND env variable)
!KERAS_BACKEND=torch python export_gemma_to_hf.py \
    --weights_file $FINETUNED_WEIGHTS_PATH \
    --size $"2b" \
    --vocab_path $FINETUNED_VOCAB_PATH \
    --output_dir $HUGGINGFACE_MODEL_DIR

2024-03-25 14:40:14 URL:https://raw.githubusercontent.com/keras-team/keras-nlp/master/tools/gemma/export_gemma_to_hf.py [11761/11761] -> "export_gemma_to_hf.py" [1]
2024-03-25 14:40:25.402439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 14:40:25.402492: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 14:40:25.403860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

-> Loading Keras weights from file `./my_model/model.weights.h5`...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 

In [21]:
from numba import cuda
import transformers

In [22]:
device = cuda.get_current_device()
cuda.select_device(device.id)
cuda.close()

In [23]:
model = transformers.GemmaForCausalLM.from_pretrained(
    HUGGINGFACE_MODEL_DIR,
    local_files_only=True,
    device_map="auto",  # Library "accelerate" to auto-select GPU
)
tokenizer = transformers.GemmaTokenizer.from_pretrained(
    HUGGINGFACE_MODEL_DIR,
    local_files_only=True,
)

OSError: Incorrect path_or_model_id: './Course_Title_Gemma_2b'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
def gen_test_prompt(key_ideas, lang='en'):
    return f"Instruction: Your task is to choose a suitable title for a course which teach these key ideas: {key_ideas}, the course's title should be in language {lang}\nResponse:"

In [ ]:
inputs = tokenizer([gen_test_prompt(key_ideas="Fundamentals of Data Science and Maths, Aritficial Intelligence")], return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=128)

output = tokenizer.decode(outputs[0], skip_special_tokens=True)
output

In [ ]:
model.push_to_hub('VietTung04/Gemma_Course_title_gen_2b')

In [ ]:
tokenizer.push_to_hub('VietTung04/Gemma_Course_title_gen_2b')